<a href="https://colab.research.google.com/github/purnawanpp/Rekap-script/blob/main/Meeko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparation ligand using Meeko**

[Meeko](https://github.com/forlilab/Meeko) is a Python package developed by the Forli group at Scripps Research, specifically designed to streamline the process of preparing molecules for docking simulations with popular software like AutoDock 4 and Vina. It simplifies the preparation of ligands and proteins by offering functionalities to handle various molecular formats, perform necessary modifications, and generate input files required for docking studies.

In [5]:
pip install rdkit-pypi

In [4]:
pip install meeko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 2.9 MB/s eta 0:00:00


In [6]:
!apt install openbabel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 4 newly installed, 0 to remove and 24 not upgraded.
Need to get 3,903 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel amd64 3.1.1+dfsg-6ubuntu5 [128 kB]
Fetched 3,903 kB in 0s (18.6 MB/s)
Selecting previously unselected package libinchi1.
(Reading database ... 121666 files and direc

**Command to add explisit hydrogen**

In [15]:
!obabel lig.sdf -h -O lig_H.sdf

1 molecule converted


In [16]:
import meeko, rdkit
from rdkit import Chem
from rdkit.Chem import rdmolops
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy

# referece: https://gist.github.com/greglandrum/7f546d1e35c2df537c68a64d887793b8
def add_formal_charges(m):
    m.UpdatePropertyCache(strict=False)
    for at in m.GetAtoms():
        # [O-]
        if at.GetAtomicNum() in [8] and at.GetExplicitValence()==1 and at.GetFormalCharge()==0:
            at.SetFormalCharge(-1)

AH_from_sdf = Chem.SDMolSupplier('/content/lig_H.sdf', removeHs=False)[0]
print("Formal Charge from Input:", rdmolops.GetFormalCharge(AH_from_sdf)) # will get -42

# Correct charge and update valence properties
add_formal_charges(AH_from_sdf)
AH_from_sdf.UpdatePropertyCache(strict=False)

print("Formal Charge after Correction:", rdmolops.GetFormalCharge(AH_from_sdf)) # will get -44

# Generate ligand pdbqt string
preparator = MoleculePreparation()
mol_setups = preparator.prepare(AH_from_sdf)
for setup in mol_setups:
    pdbqt_string = PDBQTWriterLegacy.write_string(setup)[0]

# Write pdbqt string to file
with open('lig.pdbqt','w') as fw:
    for line in pdbqt_string:
        fw.write(line)

Formal Charge from Input: 0
Formal Charge after Correction: 0


In [ ]:
!mk_prepare_ligand.py

usage: mk_prepare_ligand.py [-h] [-v] -i INPUT_MOLECULE_FILENAME [-o OUTPUT_PDBQT_FILENAME]
                            [--multimol_outdir MULTIMOL_OUTPUT_DIR]
                            [--multimol_prefix MULTIMOL_PREFIX] [-] [-c CONFIG_FILE]
                            [--rigid_macrocycles] [--keep_chorded_rings] [--keep_equivalent_rings]
                            [--min_ring_size MIN_RING_SIZE] [-w]
                            [--merge_these_atom_types [MERGE_THESE_ATOM_TYPES ...]] [-r SMARTS]
                            [-b i j [i j ...]] [-a] [-p ATOM_TYPE_SMARTS_JSON]
                            [-aa [{'smarts': '<smarts pattern>', 'atype': ',atomtype name>'}, {'smarts': '<smarts pattern>', 'atype': ',atomtype name>'}]]
                            [--double_bond_penalty DOUBLE_BOND_PENALTY] [--bad_charge_ok]
                            [--add_index_map] [--remove_smiles]
                            [--reactive_smarts REACTIVE_SMARTS]
                            [--reactive_sma

In [18]:
!mk_prepare_receptor.py

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


usage: mk_prepare_receptor.py [-h] [--pdb PDB] [--pdbqt PDBQT] -o OUTPUT_FILENAME [-f FLEXRES]
                              [-r REACTIVE_FLEXRES] [-g REACTIVE_NAME] [-s REACTIVE_NAME_SPECIFIC]
                              [--box_size BOX_SIZE BOX_SIZE BOX_SIZE]
                              [--box_center BOX_CENTER BOX_CENTER BOX_CENTER]
                              [--box_center_on_reactive_res] [--ligand LIGAND] [--padding PADDING]
                              [--skip_gpf] [--r_eq_12 R_EQ_12] [--eps_12 EPS_12]
                              [--r_eq_13_scaling R_EQ_13_SCALING]
                              [--r_eq_14_scaling R_EQ_14_SCALING]

options:
  -h, --help            show this help message and exit
  --pdb PDB             input can be PDBQT but charges and types will be reassigned
  --pdbqt PDBQT         keeps existing charges and types
  -o OUTPUT_FILENAME, --output_filename OUTPUT_FILENAME
                        adds _rigid/_flex with flexible residues. Always suffixes .